In [19]:
import fastf1 as f1
from fastf1.core import Laps
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
f1.Cache.enable_cache("C:\Windows\Temp")
session = f1.get_session(2021,"Spanish Grand Prix", "Q")

In [6]:
session.load()

core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v2.3.0]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for session_status_data. Loading data...
api            INFO 	Fetching session status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for track_status_data. Loading data...
api            INFO 	Fetching tra

In [18]:
drivers = pd.unique(session.laps["Driver"])
fastest_laps = [session.laps.pick_driver(drv).pick_fastest() for drv in drivers]
fastest_laps = Laps(fastest_laps).sort_values(by = "LapTime").reset_index(drop = True)


,Time,DriverNumber,LapTime,LapNumber,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,...,Compound,TyreLife,FreshTyre,Stint,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
0,0 days 01:18:02.424000,44,0 days 00:01:16.741000,15,NaT,NaT,0 days 00:00:21.798000,0 days 00:00:28.543000,0 days 00:00:26.400000,0 days 01:17:07.481000,...,SOFT,2.0,True,5,0 days 01:16:45.683000,Mercedes,HAM,1,True,2021-05-08 14:01:45.692
1,0 days 01:18:07.649000,33,0 days 00:01:16.777000,10,NaT,NaT,0 days 00:00:21.759000,0 days 00:00:28.723000,0 days 00:00:26.295000,0 days 01:17:12.631000,...,SOFT,2.0,True,3,0 days 01:16:50.872000,Red Bull Racing,VER,1,True,2021-05-08 14:01:50.881
2,0 days 01:18:14.348000,77,0 days 00:01:16.873000,12,NaT,NaT,0 days 00:00:21.832000,0 days 00:00:28.612000,0 days 00:00:26.429000,0 days 01:17:19.307000,...,SOFT,2.0,True,4,0 days 01:16:57.475000,Mercedes,BOT,1,True,2021-05-08 14:01:57.484
3,0 days 01:26:06.880000,16,0 days 00:01:17.510000,17,NaT,NaT,0 days 00:00:21.880000,0 days 00:00:28.822000,0 days 00:00:26.808000,0 days 01:25:11.250000,...,SOFT,2.0,True,6,0 days 01:24:49.370000,Ferrari,LEC,1,True,2021-05-08 14:09:49.379
4,0 days 01:17:36.938000,31,0 days 00:01:17.580000,11,NaT,NaT,0 days 00:00:21.861000,0 days 00:00:29.087000,0 days 00:00:26.632000,0 days 01:16:41.219000,...,SOFT,2.0,True,4,0 days 01:16:19.358000,Alpine,OCO,1,True,2021-05-08 14:01:19.367
5,0 days 01:17:21.951000,55,0 days 00:01:17.620000,11,NaT,NaT,0 days 00:00:21.942000,0 days 00:00:28.908000,0 days 00:00:26.770000,0 days 01:16:26.273000,...,SOFT,2.0,True,4,0 days 01:16:04.331000,Ferrari,SAI,1,True,2021-05-08 14:01:04.340
6,0 days 01:17:30.933000,3,0 days 00:01:17.622000,11,NaT,NaT,0 days 00:00:21.773000,0 days 00:00:29.067000,0 days 00:00:26.782000,0 days 01:16:35.084000,...,SOFT,2.0,True,4,0 days 01:16:13.311000,McLaren,RIC,1,True,2021-05-08 14:01:13.320
7,0 days 01:26:01.802000,11,0 days 00:01:17.701000,16,NaT,NaT,0 days 00:00:21.885000,0 days 00:00:28.976000,0 days 00:00:26.840000,0 days 01:25:05.986000,...,SOFT,2.0,True,5,0 days 01:24:44.101000,Red Bull Racing,PER,1,True,2021-05-08 14:09:44.110
8,0 days 00:55:10.184000,18,0 days 00:01:17.974000,8,NaT,NaT,0 days 00:00:21.925000,0 days 00:00:29.081000,0 days 00:00:26.968000,0 days 00:54:14.135000,...,SOFT,2.0,True,3,0 days 00:53:52.210000,Aston Martin,STR,1,True,2021-05-08 13:38:52.219
9,0 days 00:55:01.483000,10,0 days 00:01:17.982000,5,NaT,NaT,0 days 00:00:22.037000,0 days 00:00:29.002000,0 days 00:00:26.943000,0 days 00:54:05.538000,...,SOFT,2.0,True,2,0 days 00:53:43.501000,AlphaTauri,GAS,1,True,2021-05-08 13:38:43.510
